# Import

In [ ]:
import Foundation
import TensorFlow
import SwiftCoreMLTools

# 1. Data Ingestion

In [ ]:
let data = try String(contentsOfFile:"./data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

var index = Set<Int>()

while index.count < numRecords {
    index.insert(Int.random(in: 0..<numRecords))
}

let randomDataRecords = index.map{ dataRecords[$0] }

let dataFeatures = randomDataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = randomDataRecords.map{ Array($0[(numColumns-1)...]) }

# 2. Data Transformation

## 2.1. Split Numerical Categorical Features

In [ ]:
let categoricalColumns = [3, 8]
let numericalColumns = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]
let numCategoricalFeatures = categoricalColumns.count
let numNumericalFeatures = numericalColumns.count
let numLabels = 1

assert(numColumns == numCategoricalFeatures + numNumericalFeatures + 1)

## 2.2. Get Numerical Features

In [ ]:
let numericalFeatures = dataFeatures.map{ row in numericalColumns.map{ row[$0] } }

## 2.3. Get Categorical Features

In [ ]:
let allCategoriesValues = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
                                .reduce(into: Array(repeating: [Int32](), count: 2)){ total, value in
                                    total[0].append(value[0])
                                    total[1].append(value[1]) }
                                .map{ Set($0).sorted() }

let categoricalFeatures = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }

## 2.4. Categorize Categorical Features with Ordinal values

In [ ]:
var categoricalValues = Array(repeating: Set<Int32>(), count: 2)

for record in categoricalFeatures {
    categoricalValues[0].insert(record[0])
    categoricalValues[1].insert(record[1])
}

let sortedCategoricalValues = [categoricalValues[0].sorted(), categoricalValues[1].sorted()]

let ordinalCategoricalFeatures = categoricalFeatures.map{ [Int32(sortedCategoricalValues[0].firstIndex(of:$0[0])!), 
                                                           Int32(sortedCategoricalValues[1].firstIndex(of:$0[1])!)] }

## 2.5. Split Train and Test

In [ ]:
let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

func matrixTranspose<T>(_ matrix: [[T]]) -> [[T]] {
    if matrix.isEmpty {return matrix}
    var result = [[T]]()
    for index in 0..<matrix.first!.count {
        result.append(matrix.map{$0[index]})
    }
    return result
}

let xCategoricalAllTrain = matrixTranspose(Array(ordinalCategoricalFeatures[0..<numTrainRecords]))
let xCategoricalAllTest = matrixTranspose(Array(ordinalCategoricalFeatures[numTrainRecords...]))
print("xCategoricalAllTest:",xCategoricalAllTest)
let xNumericalAllTrain = Array(Array(numericalFeatures[0..<numTrainRecords]).joined())
let xNumericalAllTest = Array(Array(numericalFeatures[numTrainRecords...]).joined())
print("xNumericalAllTest:",xNumericalAllTest)
let yAllTrain = Array(Array(dataLabels[0..<numTrainRecords]).joined())
let yAllTest = Array(Array(dataLabels[numTrainRecords...]).joined())
print("yAllTest:",yAllTest)

let XCategoricalTrain = xCategoricalAllTrain.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTrainRecords, 1]))
}
let XCategoricalTest = xCategoricalAllTest.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTestRecords, 1]))
}

let XNumericalTrainDeNorm = Tensor<Float>(xNumericalAllTrain).reshaped(to: TensorShape([numTrainRecords, numNumericalFeatures]))
let XNumericalTestDeNorm = Tensor<Float>(xNumericalAllTest).reshaped(to: TensorShape([numTestRecords, numNumericalFeatures]))
print("XNumericalTestDeNorm:",XNumericalTestDeNorm)
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, numLabels]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, numLabels]))
print("YTest:",YTest)

## 2.6. Normalize Numerical Features

In [ ]:
let mean = XNumericalTrainDeNorm.mean(alongAxes: 0)
let std = XNumericalTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

In [ ]:
let XNumericalTrain = (XNumericalTrainDeNorm - mean)/std
let XNumericalTest = (XNumericalTestDeNorm - mean)/std

In [ ]:
print("Training shapes \(XNumericalTrain.shape) \(XCategoricalTrain[0].shape) \(XCategoricalTrain[1].shape) \(YTrain.shape)")
print("Testing shapes  \(XNumericalTest.shape) \(XCategoricalTest[0].shape) \(XCategoricalTest[1].shape) \(YTest.shape)")

## 3. Export pre-trained non-updatable model

In [ ]:
// Form a protobuf data structure
let coremlModel = Model(version: 1,
                        shortDescription: "Regression ML Model",
                        author: "Denis Simon",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.6"]) {
    Input(name: "numericalInput", shape: [11])
    Input(name: "categoricalInput1", shape: [1])
    Input(name: "categoricalInput2", shape: [1])
    Output(name: "output", shape: [1])
    NeuralNetwork {
        Embedding(name: "embedding1",
                     input: ["categoricalInput1"],
                     output: ["outEmbedding1"],
                     weight: [0.31650227, 0.7767089, 0.99034077, 0.3925201],
                     inputDim: 2,
                     outputChannels: 2)
        Permute(name: "permute1",
                     input: ["outEmbedding1"],
                     output: ["outPermute1"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten1",
                     input: ["outPermute1"],
                     output: ["outFlatten1"],
                     mode: .last)
        Embedding(name: "embedding2",
                     input: ["categoricalInput2"],
                     output: ["outEmbedding2"],
                     weight: [0.3657554, 0.7840034, -0.3617105, 0.46863696, 0.40025523, 0.67223084, 0.5171649, 1.0809144, 0.713967, 0.48226324, 0.52345484, 0.49742022, 1.0212471, -0.030412424, -0.43757045, 0.6589401, 0.53472286, 0.06093046, 1.1797707, 0.14686356, 0.52193177, 0.9100212, 1.0202348, 0.9297347, 1.2376559, 0.0719154, 0.49266085, 0.13300367, 0.83706516, 1.1322919, 0.54649436, -0.1165595, 0.5051293, 0.5876032, 0.70808876, 0.3914912, -0.20258053, -0.14893825, 1.1696767, 0.35263395, 0.8652944, 0.20865263, 0.34934682, 0.52554965, 1.0248908],
                     inputDim: 9,
                     outputChannels: 5)
        Permute(name: "permute2",
                     input: ["outEmbedding2"],
                     output: ["outPermute2"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten2",
                     input: ["outPermute2"],
                     output: ["outFlatten2"],
                     mode: .last)
        Concat(name: "concat",
                     input: ["numericalInput", "outFlatten1", "outFlatten2"],
                     output: ["outConcat"])
        InnerProduct(name: "dense1",
                     input: ["outConcat"],
                     output: ["outDense1"],
                     weight: [0.099804126, 0.10387972, 0.16136672, -0.2831368, 0.15900494, 0.08178591, -0.39829165, 0.23986651, -0.2508647, -0.016519472, -0.44759998, 0.088705085, -0.16025014, 0.063932344, 0.050376035, -0.39556986, -0.022744454, 0.11132251, -0.12420576, 0.20720538, -0.18381746, 0.021920852, 0.29230917, 0.10006229, -0.2222911, 0.16896874, 0.121601135, 0.0390108, -0.2948671, 0.21023436, 0.19271357, 0.20484433, -0.26395646, 0.25625652, 0.29149097, 0.27429712, 0.24100867, -0.31024018, -0.008066113, -0.09779309, 0.22564569, 0.71220297, -0.021357505, 0.16444775, 0.27806908, 0.15969753, 0.16671596, 0.27229562, -0.043659758, -0.15457992, 0.06595582, -0.218953, 0.2053832, -0.05260853, 0.13023151, 0.15493001, 0.09119585, -0.2565806, 0.48205203, -0.10817277, 0.08515091, -0.47849414, -0.050685298, -0.29132962, -0.07089177, -0.11379753, 0.15057145, -0.055008613, 0.18362384, 0.088105455, 0.17835003, 0.023157727, -0.3673147, 0.21326184, -0.2694485, -0.2501521, -0.09127522, 0.1894045, 0.13245964, 0.110349454, -0.569864, 0.23587716, 0.28109223, 0.13971703, 0.08089816, 0.08519786, -0.07661261, 0.11262591, 0.1292612, 0.16121916, -0.16128206, -0.008052797, -0.13941367, -0.007697845, 0.42639494, -0.36446714, -0.17010088, -0.2787905, -0.11827592, -0.10031675, 0.016368112, -0.016194269, 0.017976979, -0.1025827, -0.010503844, 0.1702527, 0.12779526, -0.18345557, -0.24336171, 0.07867038, -0.09855744, -0.27157533, 0.15600914, 0.10508666, -0.5155129, -0.3499383, 0.12571353, -0.022979988, -0.48960575, 0.5426407, 0.20436546, 0.053936522, 0.1181504, -0.087064385, 0.19482864, -0.051739406, 0.18005237, -0.26236624, -0.31114078, 0.15031318, -0.37182915, 0.12159543, -0.36398447, 0.0065399474, 0.15609597, -0.5256724, -0.2214745, -0.15480642, 0.18776892, -0.3947641, -0.24545102, -0.49781397, 0.18604112, 0.18723415, 0.17121533, 0.13131335, 0.4006924, -0.31222826, -0.60701495, -0.5603337, 0.035844512, -0.25159872, -0.4228356, -0.08503658, -0.007491978, -0.21651372, -0.11153076, -0.5051671, 0.39052942, -0.13713577, 0.23305976, 0.026965065, 0.06556098, 0.04008366, 0.09133038, -0.6012559, 0.12641436, 0.23117869, -0.10465879, 0.12895046, 0.14717482, 0.12987362, -0.37048286, 0.4715175, -0.10158986, 0.09767242, 0.035628855, -0.061686747, 0.28946725, -0.18160671, -0.11452704, 0.34468448, 0.16903728, -0.009165148, 0.4038775, 0.16090912, 0.2362932, 0.29592174, 0.06254562, 0.561382, 0.38030237, -0.1422434, -0.23637463, 0.38433233, 0.26316893, -0.10357688, -0.058087226, 0.1823687, 0.070833504, -0.48634705, 0.2153703, -0.118960805, -0.076517224, 0.23545507, -0.15833744, -0.25272223, -0.1417504, -0.3981136, -0.2714169, 0.18065809, 0.047188967, -0.14299197, -0.010601631, 0.018365417, 0.24442585, -0.21188812, -0.022850735, 0.07674607, 0.13697335, 0.16677469, 0.15201104, -0.3123536, 0.3719882, -0.2734662, -0.20136979, 0.006809721, 0.5234162, -0.15070271, 0.34463012, 0.028265525, -0.025005456, 0.31420386, 0.21886463, 0.1521122, -0.32547438, 0.020748932, 0.044787336, 0.044372857, 0.28943124, 0.30890474, -0.11541956, 0.11070122, -0.37672263, 0.2819551, -0.46933547, 0.1451143, 0.015760127, 0.34472972, 0.054677516, 0.30523527, 0.1603087, -0.052663505, -0.022011854, 0.2678539, 0.059249748, 0.0019141362, 0.34144092, -0.20284389, -0.383064, -0.12754592, 0.38372087, 0.2466396, -0.10410221, 0.29470253, -0.092844725, 0.025499487, 0.2750812, -0.25425887, 0.22432166, 0.36412373, -0.04977239, -0.03056191, -0.1218331, 0.116413146, -0.042699367, -0.025496753, -0.2559981, -0.006511777, -0.34990823, -0.027128896, -0.40276846, -0.040627368, -0.015274342, 0.04419465, -0.16637903, -0.09392845, -0.20143104, 0.32789683, 0.15741721, 0.06797972, -0.28798205, -0.136477, 0.669809, 0.37419206, 0.06428568, -0.17206271, 0.11119822, 0.8240591, 0.12439346, 0.07535555, -0.03504729, -0.12527697, 0.35121962, 0.09850336, 0.40291223, -0.03901551, -0.1741254, 0.043839857, -0.18929262, 0.1880399, 0.34359047, -0.03204634, 0.03552947, -0.23626252, -0.33345866, 0.25522295, 0.25403214, 0.049164575, 0.3448423, -0.03132553, 0.06697109, 0.007718904, -0.118182555, -0.075650506, -0.069406554, -0.039530654, 0.21440557, 0.0064389096, 0.39747772, -0.029183004, 0.07298442, -0.20115143, -0.029872585, 0.00071746274, -0.12718686, -0.11221839, -0.20320837, 0.04506928, 0.036349487, 0.06326518, -0.4165809, -0.29016638, -0.16060798, -0.09242549, -0.03147998, -0.5503854, -0.08866144, 0.19100778, -0.19292189, -0.1727462, 0.17702483, -0.08312916, -0.19610271, 0.27647278, 0.070838556, 0.12240521, -0.17649835, 0.06787486, 0.22875632, 0.1695661, 0.09826359, 0.024497965, -0.26484928, 0.26910886, -0.8564467, -0.007947221, -0.14901005, -0.0069302293, 0.41017786, -0.05145432, 0.065154776, 0.08405297, 0.03502152, 0.26342452, 0.02890973, 0.34403142, 0.44676316, -0.18308073, 0.34783784, -0.13815126, 0.2071362, -0.19769663, 0.31636894, 0.2553416, 0.35653877, 0.51120573, 0.26151508, 0.50427955, -0.016058275, -0.16041349, 0.077834256, 0.016880462, 0.117358, 0.08868193, -0.11597497, 0.34326738, 0.07044557, -0.12899244, -0.17504929, 0.3813543, 0.21503082, 0.027735956, -0.28907976, 0.1396541, -0.18675344, 0.10651747, -0.27447525, -0.104932904, 0.1471051, 0.07558783, 0.0031677806, 0.0068064816, 0.118082225, 0.1902728, 0.114664234, 0.015330532, -0.16943033, 0.1121601, 0.1197503, -0.16610661, 0.19567789, 0.08600222, -0.36250147, 0.08117462, 0.38722593, -0.11803689, -0.015794512, 0.25213835, 0.13558108, 0.012345421, 0.01841922, 0.1096428, 0.033235878, 0.118356995, 0.17953354, -0.20936626, -0.022575732, 0.20699763, 0.40217498, -0.34241986, -0.21339798, 0.42960465, 0.009621912, -0.09461576, -0.273339, -0.43695596, 0.24056937, 0.2793808, 0.09214524, -0.0018445953, 0.168204, 0.16312967, -0.06422449, -0.14704578, -0.08154928, 0.14321832, -0.335458, 0.232404, -0.17288633, 0.01147551, -0.10446038, 0.29386443, 0.042726655, 0.19613032, 0.20183702, 0.27672046, 0.2716068, 0.038814966, -0.0021346232, -0.12756662, 0.16634323, 0.29805258, 0.40243953, -0.030672764, -0.16871725, -0.29252014, -0.16357668, 0.20926113, -0.10542916, 0.07687948, 0.19143412, -0.08620009, 0.032683067, 0.22487701, -0.086400695, 0.26379508, -0.15005915, -0.03836763, 0.12900923, 0.07775497, 0.13958646, -0.22814141, -0.039353933, 0.15796119, -0.08768857, -0.4072741, -0.073895544, -0.04862546, 0.24602404, -0.062184133, -0.03746828, 0.111616455, 0.123385236, 0.16135521, -0.26603362, -0.41996613, -0.11158737, -0.088193186, -0.24940304, 0.05813364, -0.48067635, 0.1514244, 0.30402598, -0.303705, 0.13230239, -0.22577013, -0.17475033, 0.2545059, 0.22496535, -0.30480814, 0.07624326, 0.23770474, -0.10158748, 0.15895556, -0.16171665, -0.07824372, 0.39635432, 0.30360055, 0.17637232, -0.16278782, -0.15534908, 0.539522, 0.21944861, -0.07385251, -0.08633467, -0.079425275, 0.3215424, 0.21563847, -0.19097212, -0.12883238, 0.39059326, 0.30504042, 0.13470754, 0.46323597, 0.0247527, 0.23771474, -0.15732808, -0.1184141, -0.3721662, 0.24888627, 0.19876067, -0.069528855, 0.0067647616, -0.24168174, 0.038249042, -0.03288505, -0.28923145, -0.12095719, 0.14261177, -0.021461265, -0.24830723, -0.09257477, -0.39077744, 0.030257015, 0.17778037, -0.026190482, 0.39345938, -0.13228148, 0.06688902, 0.42817453, 0.045812067, 0.09795059, 0.0013652402, -0.18605225, 0.0886801, 0.0448969, -0.0795618, -0.3514512, -0.13351892, 0.090727, -0.21291202, 0.23338494, -0.5161681, 0.23390095, 0.13829537, 0.2818648, 0.2917353, -0.16570169, -0.14403185, -0.30002117, 0.02661746, -0.31560776, 0.101360545, 0.18939276, 0.007917082, -0.31763583, -0.031275854, 0.23683721, -0.1385309, 0.05967703, -0.15036212, -0.16277446, 0.5235339, 0.12392736, 0.25216976, 0.030036831, -0.20605692, -0.19258733, 0.1413569, 0.24205153, 0.23001158, -0.14925355, -0.48047683, 0.12534222, 0.18287681, 0.3096846, 0.015104717, -0.06333074, 0.036145315, 0.38721946, -0.1387375, 0.034623753, -0.41711727, 0.2576021, -0.0033233983, 0.096650414, -0.14944471, -0.38618192, 0.17403805, 0.023172481, -0.17488633, -0.22718376, -0.049578186, 0.32668757, -0.09289481, 0.27198032, -0.44924855, 0.13801338, -0.5003871, -0.55805606, -0.10691519, 0.00056036905, -0.020317432, -0.1856118, -0.4169546, -0.027978012, 0.2407171, 0.37420475, -0.14210887, 0.36495712, -0.40067685, -0.12112034, 0.18413465, 0.011831749, 0.28168648, -0.17023817, 0.39916912, -0.38975516, -0.38063493, 0.02594303, -0.29437438, -0.22199205, 0.004873618, 0.027188793, 0.011288801, -0.5891093, 0.007076596, -0.0208536, -0.14129886, -0.04131738, 0.38772693, -0.27060223, 0.21342224, 0.11619605, 0.0329904, -0.06268449, 0.03725444, -0.20714661, -0.15372723, 0.09661076, -0.02149461, -0.3390714, -0.15747395, -0.2955358, -0.3488061, 0.2639389, 0.32207334, -0.026945494, 0.42624164, -0.34170362, 0.074377015, -0.41805682, -0.013515764, -0.033625223, 0.017115902, -0.0107943835, 0.16884373, -0.18624389, 0.30963838, -0.10327286, 0.23725204, 0.14224565, -0.2701411, 0.039214663, -0.30322567, -0.017110424, 0.40256065, 0.27107126, -0.07147202, -0.027423376, 0.097710714, -0.33304945, -0.18003711, -0.3206688, -0.011453287, -0.11217614, 0.21746159, -0.021367695, -0.14041048, 0.24527404, 0.14718772, 0.22481301, -0.07587083, -0.025554556, 0.40134576, -0.043571092, -0.3455907, 0.39360696, 0.08016256, 0.22891136, -0.15653974, 0.3105066, -0.043004125, -0.19375956, -0.47809342, -0.20510729, -0.34728947, 0.06843573, 0.2789481, -0.22574928, -0.03078536, -0.19984688, 0.3359172, -0.06755895, -0.38382432, 0.26369673, -0.08854292, 0.41285145, 0.005097413, 0.18326329, -0.045538414, 0.18001291, 0.18130918, -0.004305742, -0.04834159, 0.34216258, 0.25891495, 0.41526067, 0.2162161, 0.16114418, 0.14595702, -0.04727166, -0.15971226, -0.3369928, 0.32295915, -0.43478203, 0.3296759, 0.0943765, 0.10507866, 0.07590231, 0.13553941, -0.122215465, -0.011626876, -0.088630155, 0.2640614, 0.26221824, 0.32488036, 0.18055111, 0.113323174, -0.15125549, 0.3658642, 0.024354957, 0.0685285, -0.16875984, 0.056243915, -0.0860766, 0.05775942, 0.33140588, -0.14252573, -0.22574472, 0.17331128, -0.12119208, 0.008331239, 0.010255411, 0.2992185, 0.33084923, -0.33648008, -0.11515372, -0.34333444, 0.44354263, -0.1843985, -0.005762779, -0.21324587, 0.05121728, 0.28683305, 0.2970126, 0.2828859, 0.13934141, -0.17385907, 0.4647289, 0.099472485, 0.25306952, 0.30892125, -0.24454135, 0.106893316, -0.06978246, 0.087364316, 0.049598586, -0.026219279, 0.0910891, -0.3667765, 0.12121792, 0.28409478, 0.034585252, -0.27414182, -0.18577476, -0.1547759, 0.04927949, -0.28478843, 0.28076956, -0.010855058, 0.048594285, 0.49491486, -0.42292863, -0.039731115, -0.20661028, -0.17479818, 0.12945679, 0.22670926, -0.010886977, 0.34149903, -0.15150455, 0.1600296, 0.23918758, -0.18144572, 0.16394158, 0.29094937, 0.22119007, -0.2937377, -0.3504854, 0.049116578, 0.104287855, -0.37648207, 0.13327822, -0.33822468, 0.32077745, 0.15871853, 0.11008561, 0.11574778, 0.38338703, 0.11157706, -0.0868261, 0.044720165, 0.31387937, 0.072214045, -0.30953336, -0.19500104, 0.19899508, 0.053421322, 0.09606512, 0.079321265, 0.040647335, 0.17956531, 0.21649814, -0.21144168, -0.027612042, 0.20385496, 0.18053971, 0.12936586, 0.22783984, -0.21741197, 0.35470486, -0.06275354, -0.030885378, 0.095057346, -0.29880905, 0.041903876, -0.0011499176, -0.26212335, -0.558772, -0.30685326, -0.15025935, 0.031605855, 0.23268852, -0.20249376, 0.04331021, 0.16707826, -0.23215702, -0.22666833, -0.23756026, -0.16218306, 0.23774019, -0.014266301, 0.19462068, 0.4287219, -0.07599507, 0.059087537, -0.2238209, -0.18897827, -0.004412184, -0.025337012, 0.15290868, -0.00844474, 0.019016515, 0.05955976, 0.13830696, -0.109413736, 0.08404597, 0.3006708, -0.11319718, 0.21267703, 0.2213634, 0.12948808, 0.44311017, -0.07715134, 0.23969224, 0.41704547, -0.1712575, -0.39202008, -0.2900299, 0.03626266, -0.20499484, -0.4343313, -0.26431185, -0.0077506606, 0.28062946, -0.12608382, -0.23829642, -0.06575761, -0.08920653, -0.2071535, 0.119451284, 0.030215403, 0.30643603, 0.26205033, -0.13086629, 0.08717304, -0.17563371, 0.00394026, 0.16269895, -0.20673725, 0.1547195, 0.42982948, 0.1795249, 0.047285955, -0.0048983707, -0.08924251, -0.2848888, -0.19029213, 0.30409268, -0.5324818, 0.13035676, 0.18723646, -0.2274365, -0.33034447, 0.005351838, -0.39080122, 0.106024124, -0.35025814, -0.41688824, 0.045273643, -0.09158738, -0.38070288, 0.017244346, 0.28328273, -0.26626784, -0.17122547, -0.3516319, 0.11954043, 0.015040202, 0.052789647, 0.31974202, -0.48246253, 0.3797317, -0.19365034, 0.23823753, 0.07750784, 0.180043, 0.093192674, 0.46631563, -0.24129984, -0.014239994, 0.11120134, -0.04863197, 0.026984224, -0.091535926, -0.48668283, 0.32335845, -0.11017641, -0.06538014, 0.07839196, -0.036162533, 0.08817459, 0.28726086, 0.10656754, 0.18811567, -0.2217129, 0.32101864, -0.34451216, -0.32313874, 0.14412788, 0.4023081, 0.04992086, -0.24203598, 0.38824272, -0.3135511, -0.15242864, -0.026533395, 0.1916642, 0.039401133, -0.042853072, 0.35232955, -0.17265294, 0.11903077, -0.08112999, 0.18275061, -0.05926472, -0.037662342, -0.13195132, -0.19977595, -0.04828536, 0.33810955, 0.115332074, 0.25893298, -0.2171043, 0.1710341, 0.10428043, 0.015031639, 0.17280753, 0.18378375, -0.28924438, -0.061507825, 0.3091615, 0.25402442, 0.08639523, -0.29723814, 0.01612646, -0.0013107766, 0.091534965, -0.419666, 0.23899524, -0.25891188, 0.16559005, 0.1834525, 0.3529469, 0.24582447, 0.0033768648, 0.35157835, -0.073978744, -0.06936904, -0.18602283, 0.04183992, -0.029996049, -0.15503325, -0.0032296628, -0.29689202, -0.2548938, -0.073253445, 0.076780334, -0.4068461, -0.0061179353, -0.048878074, 0.41818258, 0.03657727, 0.023485724, 0.10841182, 0.29505917, -0.25037012, 0.18945874, -0.16498506, 0.22649461, -0.13193433, -0.102027796, -0.064930975, -0.3782619, -0.016678963, -0.162031, -0.032220885, 0.13783403, 0.083676696, 0.4243725, -0.045316685, 0.36208695, 0.26487076, 0.08083117, -0.0031866638, -0.05237844, 0.2811288, -0.07494577, -0.16237357, -0.39718518, 0.1751849, -0.037196003, -0.23145846, -0.45089525, 0.20389302, 0.31414536, 0.03570309, -0.13674384, 0.36295626, -0.23929591, -0.2846316, -0.03147023, -0.27433145, 0.017666455, -0.051952098, -0.38406673, -0.115781695, 0.03347894, 0.22326387, 0.38477173, 0.21470468, -0.2191426, 0.098143116, 0.15112598, 0.08671452, -0.37497023, -0.0020313764, 0.058560077, 0.29434532, -0.044170994, 0.31034708, 0.13198906, 0.3231219, -0.4905932, 0.23631924, -0.08940243, 0.25072855, -0.03545452, -0.09594872, -0.06415222, 0.27007282, -0.11371141, 0.05583045, -0.24790245, -0.18964304, 0.2621777, 0.14143553, -0.06333392, -0.3327285, 0.026789077, 0.16821131],
                     bias: [-0.075832754, 0.07194003, 0.11845444, 0.08137766, 0.08804063, 0.06760688, 0.18963712, 0.012877924, -0.08032538, 0.07606721, -0.053454958, -0.032871533, 0.118000574, 0.14049256, 0.066545226, 0.038815364, 0.15987512, 0.047784906, -0.008701209, 0.07063359, 0.23540795, 0.032909926, 0.10335755, 0.14342952, 0.012504444, 0.05096895, 0.08155927, 0.07163819, 0.049705334, 0.15781265, -0.021823334, 0.15580627, 0.07613072, 0.04145929, -0.02419176, 0.105130635, -0.030426359, -0.067098215, 0.11265607, -0.038398918, -0.00572359, 0.068624094, -0.020590587, -0.00018460769, 0.19092657, 0.10027475, 0.13908087, 0.100765765, 0.007157799, 0.01011451, 0.06538863, 0.0014386679, 0.058779635, -0.068683825, 0.18569621, 0.10052063, 0.12184208, 0.12837377, 0.19212271, 0.08193887, 0.15081933, -0.019321356, 0.13262089, 0.067166135],
                     inputChannels: 11 + 2 + 5,
                     outputChannels: 64)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weight: [-0.42416543, -0.0644484, -0.45224014, 0.44575354, -0.057626113, -0.8291567, -0.08325516, 0.41843265, -0.5958154, -0.8337963, -0.29157162, 0.37102926, 0.18784353, 0.32471767, -0.41420823, 0.45185602, -0.3183477, -0.41305912, 0.5850897, 0.062628776, 0.02108042, -0.49390927, 0.18004799, -0.09145832, -0.8573785, 0.00024289076, -0.09979281, 0.40893823, -0.015780957, -1.2820166, 0.37361106, 0.17544474, 0.08713083, 0.14430763, 0.48464075, 0.13041414, 0.9676055, -0.11713432, -0.10597393, 0.15457493, -0.27007708, -0.4759187, 0.27245024, 0.41444555, 0.3358307, 0.28113127, -0.15300667, -0.58713686, -0.12784646, 0.6785313, 0.30393255, 0.15702564, -0.12882473, -0.13556388, -0.16098946, 0.20675349, -0.09555828, -0.04609125, 0.02976833, -0.2153323, -0.17373496, -0.07371731, 0.1746435, -0.03651883, -0.41519585, -0.074604735, -0.20726193, 0.28691468, -0.08921656, -0.09647492, -0.24706931, 0.61877, 0.3287647, -0.55190074, 0.050938215, 0.33827093, -0.11658992, -0.45884076, -0.5098322, 0.3151159, 0.080757864, 0.016451688, -0.067696, 0.116965435, 0.28743038, -0.092729636, -0.1485719, -0.029959582, 0.0481991, 0.0016416946, -0.4286711, -0.16326538, 0.37246606, -0.27784082, 0.38379872, 0.2910463, -0.05568576, 0.13746695, -0.10644574, 0.37073967, 0.7930662, 0.86382484, 0.051207755, 0.102539286, -0.6435426, -0.23185422, 0.23965718, 0.083707124, 0.31847715, -0.48618835, -0.014059709, -0.25592577, 0.10258526, -0.33339745, 0.09520594, 0.015911924, -0.0033121791, 0.47326696, -0.2778788, 0.19488476, -0.07509142, -0.48854002, -0.10364671, -0.11299892, 0.33376488, 0.3271409, -0.4443754, -0.01754461, 0.46873504, -0.09450603, -0.03516153, 0.17012724, -0.09570001, 0.1024491, 0.15066527, 0.44342855, -0.38970914, 0.4165138, -0.3905978, 0.39828146, 0.10666767, 0.05195343, 0.15651023, -0.015631888, -0.31329486, 0.016071608, 0.22676761, 0.13193144, -0.022623276, 0.29556808, 0.10133516, 0.19577697, -0.0012939066, 0.24977492, -0.14219816, 0.07528157, 0.2922047, 0.33885866, -0.39451444, 0.064185455, 0.27473512, 0.26071945, 0.29801247, 0.4305356, 0.026720408, -0.07818658, 0.041008852, 0.47644952, 0.21072616, 0.10892369, 0.012553976, -0.43427303, -0.039506078, -0.036703937, 0.37265775, -0.0336953, 0.25934058, 0.41250214, 0.19198893, 0.2288305, 0.12187076, 0.96179533, 0.088225506, 0.24581365, 0.04352758, 0.2984816, 0.113091566, 0.24961838, 0.115836546, 0.6329087, -0.011612276, 0.3977009, 0.5271124, 0.107849225, -0.19374552, 0.2990053, 0.08811188, 0.398732, 0.13906252, 0.16669698, -0.36798003, 0.3958627, -0.6346371, 0.099573724, 0.34296557, -0.03623744, 0.3487436, 0.055835962, 0.08559563, 0.18050659, 0.4870049, 0.106956884, 0.2846671, -0.015902033, 0.14316784, 0.113283545, 0.2541406, -0.11150665, -0.015967004, 0.117227376, 0.20253015, 0.37065467, -0.4401764, 0.22472025, 0.0787932, 0.07630071, 0.17685756, 0.2870029, 0.033567168, 0.059292573, 0.13750811, 0.47990355, 0.075639986, 0.0042808447, -0.28359866, -0.4186583, -0.04143078, 0.06929176, 0.14640184, 0.14643647, -0.13666916, 0.5138764, 0.28000298, 0.41799504, 0.29203406, 0.84695697, 0.06616039, -0.029047022, 0.07526484, 0.05038567, 0.40449622, 0.36076856, 0.14087881, 0.78760326, -0.031280037, 0.32527012, -0.1657026, 0.035890996, 0.09058856, 0.7344866, 0.11069024, -0.5545641, -0.051508367, 0.17859493, -0.2763032, -0.61008435, 0.24226809, -0.011223089, -0.29781154, 0.061345417, 0.12404327, -0.29374668, 0.68905926, -0.14802654, 0.45486712, -0.03509127, 0.29078358, 0.11734907, 0.12695435, -0.3150414, -0.92642117, -0.02958544, 0.31363112, 0.036656134, 0.15442343, -0.62378985, 0.4963327, 0.2576431, 0.22376235, 0.46042305, -0.44789144, 0.27904314, 0.5139275, -0.04979559, -0.61198163, 0.37578508, -0.042760905, -0.18850969, 0.14874724, 0.09169626, -0.15514448, -0.34647855, -0.5354168, -0.23739542, -0.019063205, -0.48690704, 0.14478336, 0.21060148, -0.019750915, -0.6802159, -0.8355049, -0.069321305, -0.49814418, -0.0007076313, -0.62445974, -0.29035848, 0.20605391, -0.5073402, 0.096659385, -0.60553867, -0.6318774, -0.098905526, 0.1525467, 0.6494778, -0.22398375, -0.033528797, -0.61243194, 0.031050948, 0.1527866, -0.4146257, 0.29412058, -0.4829905, -0.06422218, -0.051239714, -0.30929464, -0.1911112, 0.8221495, -0.11270712, -0.068321064, -0.30244562, 0.10046025, 0.06416858, 0.120199874, -0.49862745, -0.7895804, 0.010944005, -0.13834909, 0.05863332, 0.11686607, -0.5001632, 0.54122466, 0.26041254, 0.27141735, 0.07438052, -0.14839625, -0.03510467, 0.38583723, -0.0831074, -0.39197946, 0.22590056, -0.33446956, 0.057669584, 0.34416792, 0.13563552, -0.10687024, 0.025975976, -0.6859875, -0.123169616, 0.23799618, -0.59696925, -0.647981, 0.09605886, 0.2928226, -0.59453213, -0.046611533, 0.3147567, -0.4917634, 0.22728287, -0.24015546, -0.17393382, 0.17403439, -0.43817782, 0.05171786, -0.22405739, 0.08257329, 0.11971102, -0.25284973, -0.28911045, -0.26173034, -0.1519805, -0.4309325, -0.061698217, 0.1456123, 0.07141859, -0.11579689, -0.19651365, -0.22657123, 0.20514299, -0.10529243, 0.19453378, 0.08349439, -0.16443834, -0.16670355, -0.38499004, -0.012190104, 0.12149645, 0.2815098, -0.6416914, -0.46122387, -0.20597899, -0.01330555, 0.047207087, -0.26510078, -0.19734903, 0.0948909, 0.114727534, -0.17590049, -0.01097535, 0.13899164, -0.045213003, -0.18127699, -0.19164068, -0.4343918, 0.26416516, 0.14119658, -0.16696572, 0.27820194, 0.06754563, 0.113190874, 0.26945257, -0.5656606, -0.31976408, 0.27181992, -0.40598023, -0.046241596, 0.19065142, 0.0248531, -0.18081632, 0.4919116, 0.033622798, -0.27580035, -0.043034565, 0.039384875, 0.07709555, -0.30255058, 0.18273284, 0.29498506, 0.11066367, 0.095306575, 0.20771536, 0.13077825, -0.106787086, 0.3019448, 0.14541452, 0.18305042, 0.17667764, -0.45499474, 0.17230412, -0.479357, 0.1521312, -0.009201948, 0.1117242, 0.017948143, 0.28552148, -0.1207415, 0.30297956, 0.17789054, 0.1981336, 0.16110203, 0.02040084, 0.26489973, 0.0436753, 0.12766644, 0.059705745, 0.09382556, -0.09091663, 0.3444589, 0.35095936, -0.29256025, 0.29825473, 0.19244732, 0.06775974, 0.4074884, 0.43737748, 0.1890155, 0.028019592, 0.23021522, 0.10340021, 0.1650787, 0.43337804, -0.36097375, -0.39621875, 0.15433457, 0.22227384, 0.11581582, 0.18591298, 0.18299785, 0.30579996, 0.27775848, 0.37606192, -0.013229917, 0.82373595, -0.1010846, 0.17892253, 0.02673362, 0.28362784, 0.41340628, 0.09348849, 0.15259464, 0.49675703, 0.3159464, 0.11876219, 0.43151236, -0.0923023, 0.16272594, 0.11904921, 0.019515144, 0.12332759, 0.09623433, 0.15905227, -0.04791684, 0.3335728, -0.65839833, 0.36246008, 0.37769225, 0.27601752, 0.35529885, 0.352277, -0.28738797, 0.044374067, 0.2449358, -0.10988275, 0.18522511, 0.10775195, 0.054599524, 0.058070667, -0.012847414, 0.21203087, 0.25981778, -0.06440194, 0.041375864, 0.29830456, -0.39624667, 0.0419696, 0.08835903, 0.31886452, 0.38963148, 0.43748763, 0.2560778, 0.07551761, 0.2739273, 0.05691079, 0.048019517, 0.26667425, -0.13004154, 0.01842734, 0.14575738, -0.118761376, 0.2958323, 0.26198837, 0.15255249, 0.17567046, 0.25812232, 0.30782226, -0.098899364, 0.7092477, 0.15033409, 0.0024433674, 0.06484577, 0.2410332, -0.06992004, 0.18061869, 0.26205683, 0.51715726, 0.32509977, 0.028369827, 0.21263865, 0.22045362, 0.009402465, 0.09937939, 0.22243428, 0.38701615, 0.31030503, 0.5519473, -0.06607589, 0.41975495, -0.5054689, 0.520197, 0.07074091, -0.09080462, 0.096077114, 0.06956987, -0.20178795, 0.25939116, 0.026673663, 0.23321494, 0.18574926, 0.3844639, 0.1035487, 0.09504773, 0.26470223, 0.022681551, 0.08070128, -0.104734324, 0.378023, 0.23293002, -0.51630074, 0.066663206, 0.38332152, 0.2739318, 0.20039585, 0.33634755, 0.4294993, -0.078880444, 0.37289238, 0.4768601, 0.39092165, 0.2986423, -0.14289811, -0.050256986, -0.03995828, -0.09421564, 0.2430862, 0.25755316, 0.116957255, 0.19446617, 0.1379926, -0.03793319, 0.19191842, 0.6655265, 0.1328191, 0.15598363, 0.12926589, 0.10564566, 0.07031699, 0.050137494, 0.32031554, 0.8959137, 0.024906449, 0.20546135, 0.12715672, -0.010435269, -0.029265352, 0.088209644, -0.15742044, 0.33307078, -0.07352868, 0.33404353, -0.22419141, 0.13719763, -0.49306554, 0.15192674, 0.12697701, -0.0005454919, 0.085422486, 0.065794446, -0.30723488, 0.17805313, 0.47770298, 0.03168032, 0.03466551, 0.08827754, 0.2969529, 0.17879793, 0.289085, -0.13226078, -0.04705765, 0.22621489, 0.07560442, 0.2344451, -0.19319554, -0.04869646, 0.433644, 0.31854218, 0.09242842, 0.10369676, 0.29653352, 0.24513604, 0.13889463, 0.2580391, 0.37699118, -0.049042314, -0.3471203, -0.449217, 0.35078445, 0.1783779, 0.09174876, 0.18378192, 0.058531906, 0.5058313, 0.20456125, -0.04723636, 0.05103211, 0.5499941, 0.26539478, 0.4130681, 0.33485162, 0.22421329, 0.33282572, 0.1477984, 0.089389935, 0.4545592, -0.09051347, 0.39151895, 0.22029686, 0.025855811, 0.0395424, 0.3090431, 0.08485578, 0.005081253, 0.021368362, 0.31102413, -0.041678503, 0.091830045, -0.33836848, 0.21858723, 0.18215014, 0.16044192, 0.35215995, 0.2512369, -0.29681456, 0.32542118, 0.36588922, 0.08961422, 0.20029233, 0.37534118, -0.07509668, 0.3233829, 0.26065806, -0.014884365, 0.10689404, -0.048006844, -0.039602123, -0.025483737, -0.4103314, 0.30800837, -0.006299489, 0.22491619, 0.40278614, 0.12992005, 0.14994505, -0.016678797, 0.11703838, 0.1373684, -0.0073056417, 0.021606727, -0.36602426, -0.36869147, 0.28686646, 0.050796032, -0.04991573, -0.033217147, 0.17226884, 0.4211094, -0.046523012, 0.16491902, -0.012483704, 0.83314294, 0.1182799, 0.3319966, 0.18173935, -0.085450105, 0.2353274, 0.32626304, -0.12737538, 0.9138416, 0.39502692, 0.30676395, 0.22783631, 0.3186035, -0.03507098, 0.09305538, -0.12517077, 0.4123116, 0.2024454, 0.46741885, -0.3357691, 0.13584363, -0.2469075, 0.16739325, 0.20192653, -0.030711316, 0.1313817, 0.3484692, 0.07470385, 0.08551971, 0.1168536, 0.25574937, 0.14410245, 0.13458501, -0.031417266, 0.37795934, 0.25679764, -0.003003458, 0.21460946, -0.021286352, 0.00110467, -0.068822585, -0.5131034, 0.24692273, 0.3221286, -0.093906544, 0.084410265, 0.28813526, 0.42475733, 0.10713422, -0.049222574, 0.47214794, 0.027638737, 0.34662986, -0.30781448, -0.09448264, 0.353271, -0.08016013, 0.04203855, -0.14395663, -0.089311965, 0.44668835, -0.035939705, 0.22055136, 0.33903745, 0.7375581, -0.15289958, 0.34526205, 0.41709337, 0.08098925, 0.27235824, 0.25265047, 0.024052616, 0.85340846, 0.13368809, 0.086624995, 0.14365438, 0.27015874, -0.07340597, 0.21596526, 0.14975856, 0.2595264, 0.12585801, 0.2734626, -0.20774299, 0.049080838, -0.65496457, 0.13936432, 0.31191912, 0.1288732, 0.025488593, 0.07566803, 0.14435361, -0.030062562, 0.16271533, 0.27426526, 0.19311258, 0.19178191, 0.058216814, -0.05133697, 0.069784254, 0.28156424, -0.09759069, -0.033377334, 0.36193848, -0.036541667, -0.45884705, -0.08171312, 0.10520432, 0.18859461, 0.22476918, 0.19086142, 0.40653497, 0.1802322, 0.14127597, 0.04949609, 0.094907336, 0.35903272, -0.15612438, -0.4302053, 0.24190968, 0.11565746, 0.19374044, 0.019293612, -0.12814857, 0.2300803, 0.24727704, 0.29877484, 0.13935299, 0.60778075, 0.033037048, 0.11845389, 0.31698844, 0.2999573, 0.06768957, 0.37404588, 0.032430623, 0.538349, 0.3065847, 0.21460468, -0.039201703, -0.024231253, -0.034116887, -0.03273675, 0.026514485, 0.12816055, -0.07317659, -0.004830309, -0.07836, 0.16049589, 0.19839889, 0.14707023, -0.20085819, -0.2571293, 0.19057666, 0.05699005, 0.12804301, -0.007710426, -0.14136028, 0.0061395974, -0.19379072, -0.046303444, 0.027756887, -0.16185242, -0.24808897, -0.20849486, -0.10897743, 0.23768786, -0.07517395, -0.15100181, 0.14912732, 0.0940136, -0.12053391, 0.10686699, -0.077023, -0.13786966, -0.25148565, -0.21772736, -0.14127168, 0.027554853, 0.07962613, -0.22659566, 0.06895846, -0.06534759, -0.24456634, -0.19543208, -0.057832446, 0.056362953, -0.09371315, 0.22396012, 0.13280424, 0.058781885, 0.13767041, 0.055772215, -0.054453343, 0.06503246, 0.0060202656, -0.13523857, -0.024510924, 0.029053377, -0.09033811, 0.09777163, 0.11522925, 0.08893064, 0.43439266, 0.20891397, -0.13289103, 0.2547514, -0.053570345, 0.02148239, 0.1820483, 0.30221394, -0.27469066, 0.15439264, -0.38448027, 0.28782415, -0.007362861, 0.27173492, 0.28741494, -0.0029851587, -0.07497205, -0.12585461, 0.19415244, 0.20981689, 0.2904258, 0.13150424, 0.1863875, -0.06758074, -0.07965508, -0.039773483, 0.19563866, -0.026440596, 0.37841484, 0.055511873, -0.39774105, 0.12027766, 0.4122931, 0.21730193, 0.28339854, 0.43322214, -0.029474277, -0.083619736, 0.23432995, 0.057004794, 0.21826813, 0.38558063, -0.35787162, -0.183413, 0.12702292, 0.2795595, 0.109553136, 0.27938685, 0.052501615, 0.6268399, 0.09978659, 0.17381123, 0.28012434, 0.6273537, -0.13650341, 0.015716938, 0.44399202, 0.049616043, 0.41217285, 0.16850996, 0.31729054, 0.47415435, 0.15800145, 0.3111569, 0.016673345, 0.18029149, -0.19045095, 0.18103841, -0.054959025, 0.20135158, 0.038813163, -0.28027314, -0.014958965, -0.059964094, -0.14015429, -0.08435093, -0.002500746, -0.2059417, 0.010788201, 0.11629963, -0.11605141, 0.1655288, -0.28500566, -0.138282, 0.068145454, -0.19140136, -0.26785427, -0.12655628, -0.043583035, 0.14249338, -0.14957772, -0.2980481, -0.11746455, 0.0016986446, -0.19160022, -0.18045758, -0.052153956, -0.05973763, -0.24534003, -0.10907993, -0.26173884, -0.2569033, -0.1892535, -0.15942645, -0.1357367, 0.1356454, -0.0615032, 0.013840325, 0.025052812, -0.009087059, -0.1888852, -0.28247046, 0.17413183, 0.17644058, 0.20543692, 0.1183093, -0.065082885, 0.13066323, 0.10680129, 0.19895084, -0.25761732, 0.072649434, -0.16199234, 0.13755658, 0.08564784, -0.092402995, 0.17773601, -0.27613226, 0.44715714, 0.19095902, -0.023787023, 0.009934927, 0.25921452, -0.060354725, 0.10597158, 0.62367326, -0.18696876, 0.07062041, -0.48610914, 0.36020637, 0.24647939, -0.12089792, 0.24713089, 0.29945955, -0.24423862, 0.04354576, 0.351726, 0.3133393, 0.044721402, 0.40708324, 0.16670488, 0.32917655, 0.030220386, 0.24080476, 0.18404637, 0.33627394, -0.0067569674, -0.04505998, -0.15306267, -0.11083916, 0.11922397, -0.11765773, 0.12191703, 0.10926267, 0.3195393, 0.32818374, 0.104673006, 0.28976074, 0.3842295, 0.018471465, -0.22308008, -0.36177272, 0.24317698, 0.12163181, 0.047529966, -0.120465346, 0.23378451, 0.6267436, -0.018379662, 0.2910061, -0.060538393, 0.62211627, -0.052426085, 0.10100851, 0.14318556, 0.13077404, 0.11214588, -0.07520568, 0.15691367, 0.51425034, 0.18779722, 0.15130587, -0.59652996, 0.12289236, 0.008145498, 0.2779654, 0.17276727, -0.23981255, -0.6919175, 0.10160217, 0.401211, -0.11377504, -0.28657028, -0.5227175, -1.0197738, -0.14833395, 0.034248266, -0.15603249, 0.69838923, -0.04333509, -0.1780464, 0.0708998, -0.01508177, 0.10891175, -0.05318571, -0.87239605, 0.1354872, -0.07467028, 0.051479694, 0.1409876, 0.17899267, 0.5837544, 0.33275926, 0.28812853, -0.21520366, -0.012570253, -0.05449505, -0.15842272, 0.040474147, -0.08827428, -0.5008321, -0.097751774, -0.14349732, -0.029080858, -0.23418613, 0.18767576, 0.2293783, 0.16801034, -0.7604679, -0.18627603, 0.2554908, -0.0298032, -0.28387788, 0.093074545, 0.276348, -0.6037882, 0.32863024, 0.12563734, -0.6192619, 0.25931016, -0.7910917, -0.33506614, -0.1260619, -0.13343485, -0.13803002, 0.11501951, 0.32833907, 0.30850708, 0.33012313, -0.27193707, -0.19764408, -0.14348665, 0.35420182, 0.017103577, 0.07784605, 0.085859835, 0.009223471, 0.3084553, -0.0055278973, 0.23748066, 0.16528697, 0.3007921, -0.00013016854, 0.048201494, -0.31466866, 0.4137514, 0.019605722, -1.0088042, -0.032429755, 0.21095753, -0.29797798, 0.26344454, -0.19612695, -0.23816395, 0.18525997, 0.055599697, 0.1879435, -0.2858875, -0.40340793, -0.0361107, -0.19827655, 0.28214017, -0.035137657, -0.2018018, -0.12845272, -0.16216934, 0.42670497, -0.0035836443, 0.39099398, -0.12962155, -0.08532171, 0.24986573, 0.1590507, 0.122808866, -0.06378061, -0.50104135, 0.19581161, -0.37281838, -0.08292904, 0.42366618, 0.43682066, 0.11225371, -0.1626058, -0.38096237, -0.1320358, -0.21543147, -0.36276835, -0.7417516, 0.14372343, 0.11415928, 0.16896123, 0.23099414, -0.11121191, 0.07644544, 0.29581797, -0.09294234, 0.3198651, 0.2448835, 0.005223729, 0.12482591, -0.22992173, 0.22426398, -0.058419935, 0.008953987, 0.09109788, 0.024297303, -0.15932891, 0.21023175, 0.120309755, 0.20868194, 0.028116737, 0.06585665, 0.26577982, 0.28288683, 0.092698, -0.08346469, 0.29917622, 0.051682197, 0.0516797, 0.12365743, -0.23999204, -0.034983337, 0.4119693, -0.12176571, 0.36839938, 0.020618904, 0.09128562, 0.005180486, 0.30108327, 0.21223012, 0.47641116, 0.25163317, 0.049596164, -0.07627447, 0.31564206, 0.048348177, 0.1275683, -0.13035397, 0.3269668, 0.5486536, 0.027827635, 0.040104758, 0.29325315, 0.6593802, 0.23694, 0.14794208, 0.10621589, 0.27007952, 0.082023606, 0.0127678085, 0.11117267, 0.6772686, 0.35788113, -0.019541096, 0.22575654, 0.08944858, -0.07933746, 0.33004576, 0.21741734, 0.13684203, 0.20580132, 0.3261315, -0.19137813, 0.031921472, -0.6545857, 0.17896217, 0.4111354, 0.24183214, 0.073189825, 0.07301044, -0.13703778, 0.022363404, 0.28400102, -0.12618087, -0.10789888, 0.080473304, 0.2532573, 0.111459866, 0.17490457, -0.09725335, 0.30537584, 0.14925455, 0.07752854, 0.19260706, -0.2509798, 0.24379683, 0.35939336, 0.15365788, 0.18843858, 0.34689888, 0.23532806, -0.10082614, 0.37556726, 0.2456024, 0.21970664, 0.38785118, -0.21435152, -0.33005905, 0.2750479, 0.24526027, 0.3626732, 0.043171406, 0.1529458, 0.29675782, -0.034188088, 0.33958012, 0.17599085, 0.59493655, -0.0005619369, 0.15610676, 0.4298673, 0.15781447, 0.24198925, -0.08221274, 0.2307189, 0.619095, 0.23399381, 0.20604178, 0.50549203, 0.08768033, -0.006003938, 0.23811096, -0.061604552, -0.018210338, 0.17619957, 0.40173864, -0.36817074, 0.43351394, -0.28488138, 0.31446457, 0.118888065, 0.31495962, 0.35806835, 0.18313356, -0.103307396, 0.04904677, 0.3358299, 0.13858458, 0.03862486, 0.40158984, 0.2785041, 0.1937726, 0.0048751486, 0.062035587, 0.08939514, -0.05756632, 0.19080009, 0.28951365, -0.35517487, 0.28646967, 0.3077012, -0.015032238, 0.36413628, 0.44023603, 0.35123706, 0.18291783, 0.26237872, 0.3222398, 0.43437275, 0.028752923, 0.046014145, -0.44524613, 0.2542258, 0.15628983, -0.03552843, -0.043717526, 0.21065848, 0.36271495, 0.039255712, 0.25922123, 0.085554875, 0.6206158, 0.088404305, 0.14231724, 0.0013539926, 0.17328462, 0.072331734, 0.27742958, 0.010568124, 0.8409427, 0.12838647, 0.3858542, 0.2581013, 0.13918623, -0.16650659, 0.021936636, 0.013394934, -0.07814887, 0.10685575, 0.4513455, -0.012008441, 0.41409308, -0.47473633, 0.31360498, 0.021407854, -0.11095907, 0.4789138, 0.06426825, -0.057656907, -0.12517323, 0.41773808, 0.049377345, -0.040236328, 0.14621909, 0.35696554, 0.07191796, 0.28941393, 0.13712664, -0.061502155, -0.029464332, 0.21271595, -0.04147182, -0.5583648, 0.039719693, 0.44066733, -0.0376515, 0.4778308, 0.14937137, 0.41267973, 0.33764103, 0.21531053, 0.106963195, 0.32451558, 0.22882067, -0.23723476, -0.43027362, 0.29702982, 0.18053351, 0.008307264, 0.13784525, 0.10838102, 0.15978971, -0.115878105, 0.20685655, 0.12834132, 1.0001824, 0.0103264535, 0.27685726, 0.35018402, -0.0055800523, 0.0332008, 0.3766097, 0.08305928, 0.91937894, 0.13857783, 0.0034474516, -0.014097182, -0.14716746, 0.2706321, 0.03678651, 0.33434874, -0.02676795, -0.1438417, -0.12482841, 0.5848792, -0.6633053, 0.36996907, -0.19148684, 0.14521967, 0.15006687, -0.5681075, -0.067607045, 0.35624444, 0.11186536, -0.13018066, -0.4890087, 0.16472927, -0.07438476, -0.10510731, 0.017516052, 0.30058315, 0.27360514, 0.11674726, 0.0054827547, -0.09079409, 0.23353119, 0.55221874, -0.34023643, 0.0692881, -0.025278326, -0.046918552, 0.41917986, -0.396954, 0.05673189, 0.1966569, -0.33008793, 0.30571118, -0.19173436, 0.3564296, -0.018150344, 0.15321118, -0.35615778, -0.1901742, 0.024331765, 0.23864777, 0.15846422, 0.30095226, 0.16779135, 0.25052765, -0.08613694, -0.27408352, -0.1895298, 0.011762345, -0.094186984, -0.36488488, 0.49323535, 0.15150136, -0.14435221, -0.2259506, -0.22760186, 0.38778403, 0.1623746, 0.13596198, 0.114736475, -0.1014092, 0.32029083, -0.006813367, 0.29265916, -0.15240753, 0.4381529, -0.54127765, 0.17503083, 0.3452335, 0.23680814, 0.36277965, 0.03255162, -0.19393064, 0.17602354, 0.33288738, 0.2316315, -0.04394071, 0.1046879, 0.25852212, 0.107009046, 0.019654345, 0.12394397, 0.13166668, 0.32957745, -0.018040963, -0.0062040803, -0.24009795, 0.10724387, 0.44247052, -0.09850413, 0.0798399, 0.011826767, 0.32033813, -0.030981299, 0.29127228, 0.03315933, 0.40489992, 0.29349247, -0.07785271, -0.03118256, 0.17579308, 0.025302667, 0.27863017, -0.11094357, 0.13698445, 0.27886853, 0.09772593, 0.15653557, 0.18047024, 0.74583286, 0.015879562, 0.3917622, 0.16073741, -0.012330398, 0.21149161, 0.270909, -0.1029346, 0.62632555, -0.034049194, 0.14613464, 0.1692222, 0.100210294, 0.17782998, 0.22381994, 0.20522149, 0.09299883, 0.33290777, 0.56585795, -0.19782211, -0.0064947354, -0.6539411, 0.060572803, 0.1559177, 0.1533616, 0.17508377, 0.28865007, -0.066532195, 0.07873812, 0.21875058, 0.12496223, 0.008909202, 0.021482306, 0.05668321, 0.08265659, 0.17544082, 0.20501214, 0.27987066, 0.23748082, 0.053576328, 0.2669853, -0.43123174, 0.0022285534, 0.05437708, 0.07823193, 0.069861285, 0.12767881, 0.02003887, 0.3934556, 0.30362082, 0.47041792, 0.30886063, 0.36888176, -0.24804208, -0.3127063, -0.04259835, -0.084205285, 0.008710973, 0.23731719, -0.10012406, 0.58049244, -0.06281678, 0.35396087, 0.042786032, 0.816087, -0.2039535, 0.19808269, 0.23095915, 0.017556595, 0.37388566, 0.21953247, 0.1569492, 0.48063985, 0.14175044, 0.0064765806, -0.17134243, 0.102266796, -0.24887916, -0.19582321, 0.15528634, 0.19797385, -0.12609735, 0.16176316, -0.010808269, -0.067065045, -0.17255114, 0.100022376, 0.045409627, -0.25168204, 0.19479927, -0.22962528, -0.13830902, -0.21670966, -0.24447314, 0.124038346, -0.0071696374, -0.1374507, -0.23871434, -0.2249919, -0.23473248, 0.21077311, 0.06889786, -0.15922333, -0.2158548, 0.217801, 0.21722168, -0.21044093, -0.24996619, -0.03942699, 0.1497225, -0.23708774, -0.10748283, -0.061693206, -0.115360394, -0.15283594, 0.050877575, 0.07378775, -0.17628953, 0.111422054, -0.21821138, 0.19353883, -0.06709487, -0.18957414, 0.1928988, 0.114113055, 0.015978437, 0.180187, 0.15211393, 0.12776905, -0.013817183, 0.0767527, -0.15072897, 0.19837672, -0.16793351, 0.18677148, -0.060491633, -0.1686884, -0.17985734, 0.059982635, -0.5262337, 0.20211914, -0.08967879, 0.9498333, 0.09145115, 0.0050852974, -0.3639522, -0.16061553, 0.07795128, -0.46799535, 0.2820222, -0.59266704, -0.25135362, -0.052749626, -0.26778445, -0.15674774, 0.61260337, -0.10148943, 0.10074362, -0.2493249, 0.27921072, 0.052310765, 0.11981411, -0.32137856, -0.5040364, 0.15459858, -0.007665181, -0.053241394, 0.06379721, -0.52734053, 0.50435275, 0.40313897, 0.07549615, 0.2596105, -0.31840158, -0.2113021, -0.1883141, 0.21484374, -0.11664141, 0.04895643, -0.19194287, 0.13521147, 0.08898423, 0.23956658, 0.2171446, -0.33081886, -0.4882742, 0.18262897, 0.026868956, -0.5637173, -0.7623421, 0.17441793, 0.046983276, -0.712245, -0.20167814, 0.18057005, -0.603953, 0.15935981, -0.3533693, -0.17076305, 0.029363774, 0.042106282, -0.09485992, -0.31711075, -0.28259718, -0.044851325, -0.020001234, 0.0017550625, 0.36522922, -0.25512785, -0.013310306, -0.38611165, -0.6083393, 0.22947, 0.5579892, -0.4883019, -0.57987165, 0.4117854, -0.35331777, -0.25147915, 0.39961094, 0.04940073, -0.4788941, -0.05145135, 0.29958767, 0.2805718, 0.24526224, -0.15627345, -0.37599832, 0.1599108, 0.107693955, -0.2240224, -0.06450081, 0.3096707, 0.38411462, -0.05327173, 0.2192345, 0.31759158, 0.21627389, -0.1524981, -0.25136015, -0.10626081, -0.53740495, 0.1337756, -0.08108833, 0.11069529, -0.22099109, 0.21641283, 0.058822338, -0.19962685, -0.041489094, 0.41784006, 0.10263695, 0.2527815, -0.037653945, -0.24190068, 0.10152298, -0.7194601, -0.06557785, 0.20477754, -0.19778267, 0.29809657, -0.5036475, -0.54743654, 0.19058672, -0.27685136, 0.20192626, 0.23261108, 0.257729, -0.00846927, -0.121506736, 0.3035928, 0.030942705, 0.09482323, 0.16702896, 0.63023686, -0.18517707, 0.23559962, -0.3614776, 0.47788292, 0.4070775, -0.12311782, 0.26132333, 0.08799604, -0.22714074, 0.31605273, 0.1335371, 0.28925726, -0.0421974, 0.15006618, 0.23040967, 0.22898975, -0.01273815, -0.06430413, -0.1376153, -0.11622861, 0.30325824, 0.28843188, -0.09309267, 0.29454824, 0.32881355, -0.15305768, 0.28480276, 0.16651253, 0.2332182, 0.23545648, -0.07162611, 0.07271566, 0.45728922, 0.36812288, -0.3137605, -0.27022445, 0.28554657, 0.16788885, 0.37806818, -0.10042606, 0.32725045, 0.1996987, 0.06262037, 0.16182348, 0.19892952, 0.6928754, -0.19376646, 0.15564819, -0.0023738684, 0.30596358, 0.24183393, 0.24463925, -0.0195677, 0.754374, 0.09391244, 0.10565346, -0.76168555, 0.31871814, -0.2731017, 1.0137312, -1.5493382, 0.5692753, -0.42545047, 0.1176137, 0.64471704, -1.0396309, 0.18130407, -0.20988727, -0.27830327, -0.17831697, -0.20759188, -0.048237246, 0.29349297, -0.9477875, 0.19780494, -0.107925326, 0.25682053, -0.018432155, 0.23191278, -0.73101425, -0.12994839, -0.04137817, -0.059318334, 0.28587592, 0.30503705, -0.31550333, 0.06157644, 0.03136072, 0.24092525, -0.050570022, 0.25841707, 0.27027705, 0.96980464, 0.17507951, 0.16970533, -0.10919254, -0.22758847, -0.30971038, 0.048207123, 0.040092677, 0.29680556, -0.18000123, -0.6338998, -0.37286964, 0.01634757, 0.24356349, -0.31730488, 0.07210271, 0.1509298, -0.29937428, -0.46944326, 0.19987607, -0.7011236, -0.2036383, -0.8405438, -0.4749176, 0.066258386, -0.25591084, -0.013618828, 0.32379025],
                     bias: [0.07903663, 0.030866146, 0.10373971, 0.10218481, 0.13688986, 0.1087709, 0.015522402, 0.09330976, 0.092191696, 0.090994135, 0.10457051, 0.10092749, 0.09866336, 0.095161304, -0.014076156, 0.09492916, -0.044898413, 0.096195646, 0.013465355, 0.023048993, 0.09148142, 0.09244194, 0.09367459, 0.09995778, 0.04590115, 0.09292165, 0.103160635, -0.03592354, 0.085304506, 0.028931366, 0.10059409, 0.05963512],
                     inputChannels: 64,
                     outputChannels: 32)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weight: [-1.1991159, -1.0824633, 0.3534417, 0.6269976, -0.83521056, -0.5447894, -1.0525472, 0.5505506, 0.47615972, 0.3550919, 0.63946515, 0.29801297, 0.41789058, 0.48121396, -0.38900682, 0.4327798, -0.31000006, 0.25389498, -2.0748613, -0.7339764, 0.38630584, 0.3941489, 0.2830599, 0.625189, -0.98490757, 0.35369536, 0.5486228, -0.37948638, -0.56371844, -1.0306523, 0.28183803, -1.2639261],
                     bias: [0.09408202],
                     inputChannels: 32,
                     outputChannels: 1)
    }
}

In [ ]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "./s4tf_pre_trained_model.mlmodel"))

## 4. Export updatable model

In [ ]:
// Form a protobuf data structure
let coremlModel = Model(version: 1,
                        shortDescription: "Updatable Regression ML Model",
                        author: "Denis Simon",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.6"]) {
    Input(name: "numericalInput", shape: [11])
    Input(name: "categoricalInput1", shape: [1])
    Input(name: "categoricalInput2", shape: [1])
    Output(name: "output", shape: [1])
    TrainingInput(name: "numericalInput", shape: [11])
    TrainingInput(name: "categoricalInput1", shape: [1])
    TrainingInput(name: "categoricalInput2", shape: [1])
    TrainingInput(name: "output_true", shape: [1])
    NeuralNetwork(losses: [MSE(name: "lossLayer",
                               input: "output",
                               target: "output_true")],
                  optimizer: SGD(learningRateDefault: 0.001,
                                 learningRateMax: 0.3,
                                 miniBatchSizeDefault: 32,
                                 miniBatchSizeRange: [32],
                                 momentumDefault: 0,
                                 momentumMax: 1.0),
                  epochDefault: 500,
                  epochSet: [500],
                  shuffle: true) {
        Embedding(name: "embedding1",
                     input: ["categoricalInput1"],
                     output: ["outEmbedding1"],
                     weight: [0.31650227, 0.7767089, 0.99034077, 0.3925201],
                     inputDim: 2,
                     outputChannels: 2)
        Permute(name: "permute1",
                     input: ["outEmbedding1"],
                     output: ["outPermute1"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten1",
                     input: ["outPermute1"],
                     output: ["outFlatten1"],
                     mode: .last)
        Embedding(name: "embedding2",
                     input: ["categoricalInput2"],
                     output: ["outEmbedding2"],
                     weight: [0.3657554, 0.7840034, -0.3617105, 0.46863696, 0.40025523, 0.67223084, 0.5171649, 1.0809144, 0.713967, 0.48226324, 0.52345484, 0.49742022, 1.0212471, -0.030412424, -0.43757045, 0.6589401, 0.53472286, 0.06093046, 1.1797707, 0.14686356, 0.52193177, 0.9100212, 1.0202348, 0.9297347, 1.2376559, 0.0719154, 0.49266085, 0.13300367, 0.83706516, 1.1322919, 0.54649436, -0.1165595, 0.5051293, 0.5876032, 0.70808876, 0.3914912, -0.20258053, -0.14893825, 1.1696767, 0.35263395, 0.8652944, 0.20865263, 0.34934682, 0.52554965, 1.0248908],
                     inputDim: 9,
                     outputChannels: 5)
        Permute(name: "permute2",
                     input: ["outEmbedding2"],
                     output: ["outPermute2"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten2",
                     input: ["outPermute2"],
                     output: ["outFlatten2"],
                     mode: .last)
        Concat(name: "concat",
                     input: ["numericalInput", "outFlatten1", "outFlatten2"],
                     output: ["outConcat"])
        InnerProduct(name: "dense1",
                     input: ["outConcat"],
                     output: ["outDense1"],
                     inputChannels: 11 + 2 + 5,
                     outputChannels: 64,
                     updatable: true)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     inputChannels: 64,
                     outputChannels: 32,
                     updatable: true)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     inputChannels: 32,
                     outputChannels: 1,
                     updatable: true)
    }
}


In [ ]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "./s4tf_updatable_model.mlmodel"))